This notebook explores the ocean model simulation used as an input to the generator.
We make use of the Southern Ocean Regionally Refined Mesh v 2.1 run of the E3SM / MPAS-Ocean model, a 1000 year simulation of the ocean circulation, provided at a monthly resolution.
The notebook uses `dask` to chunk this dataset throughout the workflow to enable scalable computation.

In [ ]:
import sys
import os
os.environ['USE_PYGEOS'] = '0'
import gc
import collections

import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import geopandas as gpd

import numpy as np
import xarray as xr
from xeofs.xarray import EOF
import rioxarray

import dask
import distributed

import scipy
from scipy import signal
import cftime
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd

In [ ]:
client = distributed.Client()
client

In [ ]:
# File path directories

inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'
DIR_external = 'data/external/'
DIR_interim = 'data/interim/'

# DATASET FILEPATHS
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'

# INTERIM GENERATED FILEPATHS
DIR_SORRMv21_Interim = 'data/interim/SORRMv2.1.ISMF/iceShelves_dedraft/'

# DATA FILENAMES
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'

# Ice shelf basin/catchment definitions
FILE_iceShelvesShape = 'iceShelves.geojson'

In [ ]:
SORRMv21 = xr.open_dataset(inDirName+DIR_SORRMv21+FILE_SORRMv21, chunks={"Time":36})

ICESHELVES_MASK = gpd.read_file(inDirName+DIR_external+FILE_iceShelvesShape)
icems = ICESHELVES_MASK.to_crs({'init': 'epsg:3031'});
crs = ccrs.SouthPolarStereo();

In [ ]:
flux = SORRMv21.timeMonthly_avg_landIceFreshwaterFlux
# ssh = SORRMv21.timeMonthly_avg_ssh

### Detrend and deseasonalize data

In [ ]:
# Linear trend, if any
# Debug for dask implementation, a "consistent source of headaches".
# https://ncar.github.io/esds/posts/2022/dask-debug-detrend/

def detrend_dim(data, dim, deg):
    # detrend along a single dimension
    p = data.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(data[dim], p.polyfit_coefficients)
    return data - fit

In [ ]:
# Deseasonalize
# Remove climatologies to isolate anomalies / deseasonalize
def deseasonalize(data):
    data_month = data.groupby("Time.month")
    data_clm = data_month.mean("Time") # Climatologies
    data_anm = data_month - data_clm # Deseasonalized anomalies
    return data_anm

In [ ]:
flux_detrend = detrend_dim(flux,"Time",1)
# flux_clean = deseasonalize(flux_detrend)

In [ ]:
# flux_detrend = flux_detrend.compute()

### Temporal Standard Deviation

In [ ]:
# Standard deviation in time
flux_std = flux.std('Time').compute()
flux_std.where(flux_std!=0).plot()

### Temporal Mean

In [ ]:
# Time mean
flux_tmean = flux.mean('Time').compute()
flux_tmean.where(flux_std!=0).plot()

### Cumulative melt rate (across the ice sheet)

In [ ]:
flux_ts = flux.sum(['x','y']).compute()

In [ ]:
plt.figure(figsize=(25,8))
flux_ts.plot()
plt.xlabel('Time (Simulation years)')
plt.title('Freshwater Flux - AIS Cumulative')

### Mean freshwater flux in each catchment

In [ ]:
def clip_data(total_data, basin):
    clipped_data = total_data.rio.clip(icems.loc[[basin],'geometry'].apply(mapping))
    #clipped_data = clipped_data.dropna('time',how='all')
    #clipped_data = clipped_data.dropna('y',how='all')
    #clipped_data = clipped_data.dropna('x',how='all')
    # clipped_data = clipped_data.drop("month")
    return clipped_data

In [ ]:
mean_flux = np.empty(len(icems[33:133]))

for i in range(33,133):
    clip_ds = clip_data(flux_tmean, i)
    mean_flux[i-33] = clip_ds.mean()

In [ ]:
# flux_tmean.rio.write_crs("epsg:3031",inplace=True);

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(mean_flux, marker='x', lw=0.0)

